In [3]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_excel("E:/data/Online Retail.xlsx")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


It took a few monutes to load the data, so I keep a copy as a backup.

In [6]:
df1 = df.copy(deep=True)

We need to do santity check about the data

In [7]:
df1.Country.nunique()

38

In [8]:
df1.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [9]:
customer_country=df1[['Country','CustomerID']].drop_duplicates()

customer_country.groupby(['Country'])['CustomerID'].agg('count').reset_index().sort_values('CustomerID', ascending=False)

,Country,CustomerID
36,United Kingdom,3950
14,Germany,95
13,France,87
31,Spain,31
3,Belgium,25
33,Switzerland,21
27,Portugal,19
19,Italy,15
12,Finland,12
1,Austria,11


More than 90% of the customers in the data are from United Kingdom, There’s some research indicating that customer clusters vary by geography, so here I’ll restrict the data to United Kingdom only.

In [10]:
df1 = df1[df1['Country'] == 'United Kingdom']

Check whether there are missing values in each column.

There are 133600 missing values in CustomerID column, since our analysis is based on customers, we will remove these missing values.

In [12]:
df1.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     133600
Country             0
dtype: int64

In [16]:
df1 = df1[pd.notnull(df1['CustomerID'])]
df1.isnull().sum(axis=0)

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

Check the min and max values in Unit price column

In [18]:
df1.describe(include='all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
InvoiceNo,361878,19857,576339,542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StockCode,361878,3661,85123A,1987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,361878,3860,WHITE HANGING HEART T-LIGHT HOLDER,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,361878,NaN,NaN,NaN,NaN,NaN,11.077,263.129,-80995,2,4,12,80995
InvoiceDate,361878,18441,2011-11-14 15:27:00,543,2010-12-01 08:26:00,2011-12-09 12:49:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UnitPrice,361878,NaN,NaN,NaN,NaN,NaN,3.25601,70.6547,0,1.25,1.95,3.75,38970
CustomerID,361878,NaN,NaN,NaN,NaN,NaN,15547.9,1594.4,12346,14194,15514,16931,18287
Country,361878,1,United Kingdom,361878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Remove the negative values in Quantity column

In [19]:
df1 = df1[(df1['Quantity']>0)]
df1.Quantity.min()

1

After cleaning up, we now dealing with 354345 rows and 8 columns

In [20]:
df1.shape

(354345, 8)

In [21]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 354345 entries, 0 to 541893
Data columns (total 8 columns):
InvoiceNo      354345 non-null object
StockCode      354345 non-null object
Description    354345 non-null object
Quantity       354345 non-null int64
InvoiceDate    354345 non-null datetime64[ns]
UnitPrice      354345 non-null float64
CustomerID     354345 non-null float64
Country        354345 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


Check unique value for each column

In [24]:
df1.describe(include='all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
InvoiceNo,354345,16649,576339,542,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
StockCode,354345,3645,85123A,1947,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Description,354345,3844,WHITE HANGING HEART T-LIGHT HOLDER,1940,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Quantity,354345,NaN,NaN,NaN,NaN,NaN,12.0489,190.428,1,2,4,12,80995
InvoiceDate,354345,15615,2011-11-14 15:27:00,542,2010-12-01 08:26:00,2011-12-09 12:49:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UnitPrice,354345,NaN,NaN,NaN,NaN,NaN,2.96379,17.8621,0,1.25,1.95,3.75,8142.75
CustomerID,354345,NaN,NaN,NaN,NaN,NaN,15552.4,1594.55,12346,14194,15522,16931,18287
Country,354345,1,United Kingdom,354345,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Add a column for total price

In [26]:
df1['TotalPrice'] = df1['Quantity'] * df1['UnitPrice']
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


Find out first and last order date in the data

In [27]:
df1['InvoiceDate'].min()
df1['InvoiceDate'].max()

Timestamp('2010-12-01 08:26:00')

Timestamp('2011-12-09 12:49:00')

Since recency is calculated for a point in time. The last invoice date is 2011-12-09, this is the date we will use to calculate recency.

In [28]:
import datetime as dt
NOW = dt.datetime(2011,12,10)

Create a RFM table

In [29]:
rfmTable = df1.groupby('CustomerID').agg({'InvoiceDate': lambda x: (NOW - x.max()).days, # Recency
                                        'InvoiceNo': lambda x: len(x),      # Frequency
                                        'TotalPrice': lambda x: x.sum()}) # Monetary Value

rfmTable['InvoiceDate'] = rfmTable['InvoiceDate'].astype(int)
rfmTable.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'TotalPrice': 'monetary_value'}, inplace=True)

Calculate RFM metrics for each customer

In [30]:
rfmTable.head()

,recency,frequency,monetary_value
CustomerID,,,
12346.0,325,1,77183.60
12747.0,2,103,4196.01
12748.0,0,4596,33719.73
12749.0,3,199,4090.88
12820.0,3,59,942.34


Interpretation:

CustomerID 12346 has frequency:1, monetary value:$77183.60 and recency:324 days.

CustomerID 12747 has frequency: 103, monetary value: $4196.01 and recency: 1 day

Let's check the details of the first customer.

In [31]:
first_customer = df1[df1['CustomerID']== 12346.0]
first_customer

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,77183.6


The first customer has shopped only once, bought one item at a huge quantity(74215). The unit price is very low, seems a clearance sale.

In [34]:
(NOW - dt.datetime(2011,1,18)).days==326

True

The easies way to split metrics into segments is by using quartile. 

1. This gives us a starting point for detailed analysis
2. 4 segments are easy to understand and explain

In [35]:
quantiles = rfmTable.quantile(q=[0.25,0.5,0.75])
quantiles

,recency,frequency,monetary_value
0.25,17.0,17.0,300.04
0.50,50.0,41.0,651.82
0.75,142.0,99.0,1575.89


In [36]:
quantiles = quantiles.to_dict()
quantiles

{'recency': {0.25: 17.0, 0.5: 50.0, 0.75: 142.0},
 'frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 99.0},
 'monetary_value': {0.25: 300.03999999999996,
  0.5: 651.8199999999999,
  0.75: 1575.89}}

Create a segmented RFM table

In [37]:
segmented_rfm = rfmTable

Lowest recency, highest frequency and monetary are our best customers 

In [38]:
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4
    
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1

In [39]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))

Add segment numbers to the RFM table

In [40]:
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,325,1,77183.60,4,4,1
12747.0,2,103,4196.01,1,1,1
12748.0,0,4596,33719.73,1,1,1
12749.0,3,199,4090.88,1,1,1
12820.0,3,59,942.34,1,2,2


RFM segments split your customer base into an imaginary 3D cube. It is hard to visualize. However, we can sort it out.

Add a new column to combine RFM score, 111 is the highest score as we determined earlier.

In [41]:
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) \
                            + segmented_rfm.f_quartile.map(str) \
                            + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
12346.0,325,1,77183.60,4,4,1,441
12747.0,2,103,4196.01,1,1,1,111
12748.0,0,4596,33719.73,1,1,1,111
12749.0,3,199,4090.88,1,1,1,111
12820.0,3,59,942.34,1,2,2,122


Apparently, the first customer is not our best customer at all.

Here is top 10 of our best customers!

In [42]:
segmented_rfm[segmented_rfm['RFMScore']=='111'].sort_values('monetary_value', ascending=False).head(10)

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
CustomerID,,,,,,,
18102.0,0,431,259657.30,1,1,1,111
17450.0,8,337,194550.79,1,1,1,111
17511.0,2,963,91062.38,1,1,1,111
16684.0,4,277,66653.56,1,1,1,111
14096.0,4,5111,65164.79,1,1,1,111
13694.0,3,568,65039.62,1,1,1,111
15311.0,0,2379,60767.90,1,1,1,111
13089.0,2,1818,58825.83,1,1,1,111
15769.0,7,130,56252.72,1,1,1,111
